In [4]:
#Pretend to be a .
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36", "X-Requested-With": "XMLHttpRequest"}

#Import needed modules.
import time
import pytz
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime

#Printing out today's date and the current time on the East Coast.
current_time = datetime.now()
local_time = datetime.strftime(current_time, '%b %d, %Y')
eastern_time = datetime.strftime(current_time.astimezone(pytz.timezone('US/Eastern')), '%I:%M%p')
dt_east = int(datetime.strftime(current_time.astimezone(pytz.timezone('US/Eastern')), '%H%M'))
print("Hello! Today's date is: " + str(local_time))
print("Time on the East Coast is currently: " + eastern_time)

#This logic block dictates whether the market is closed or open.
if 930 <= dt_east <= 1600:
    print("The market is open!")
else:
    print("The market is closed")

#Choose your ticker. Takes input.
ticker = input("Please choose a ticker symbol: ")

#Fun little loading bar, completely unessesary.
for i in tqdm (range (100), desc = "Loading data for " + ticker.upper()):
    time.sleep(0.01)
    
#The below try/except will attempt to run everything in "try" as long as the chosen ticker exists. If the ticker does not exist, "except" will run and print the statement.
try:
    #Selecting the table to pull data from.
    url = ('https://finviz.com/quote.ashx?t=' + ticker.upper())
    req = requests.get(url, headers = headers)
    table = pd.read_html(req.text, attrs = {"class":"snapshot-table2"})
    df = table[0]
    df2 = table[0]
    
    #Pulls company name.
    soup = BeautifulSoup(req.text, 'html.parser')
    for title in soup.find_all('title'):
        split = title.get_text()
        split2 = split.split()
        company = split2[1] + ' ' + split2[2] + ' ' + split2[3]
        company2 = split2[1] + ' ' + split2[2]
        if split2[3] != 'Stock':
            print("Company/ETF name: " + company)
        else:
            print("Company/ETF name: " + company2)
        
    #This will pull the sector of the company.
    req2 = requests.get(url, headers = headers)
    table2 = pd.read_html(req2.text, attrs = {"class":"fullview-title"})
    df3 = table2[0]
    df4 = df3[0][2]
    sector_split = df4.split('|')
    sector = sector_split[0]
    print("Sector: " + sector)
    
    #Ratios and metrics to be checked.
    data = {
        df[10][10]: ["$" + df[11][10]], #Stock Price
        df[2][0]: [df[3][0]], #P/E - under 20.0
        df[2][2]: [df[3][2]], #PEG - under 1.0
        df[2][4]: [df[3][4]], #P/B - under 1.0
        df[2][6]: [df[3][6]], #P/FCF
        df[0][6]: [df[1][6]], #Annual Dividend
        df[0][7]: [df[1][7]], #Dividend Percent
        df[6][5]: [df[7][5]]  #ROE
    }
    
    #Framing the table and printing the results.
    df = pd.DataFrame(data, index = ["Stats:"])
    print(df)
    
    try:
        #Math section which determines how many shares you'll need to buy a whole new share with only dividends being reinvested each quarter.
        price = float(df2[11][10])
        dividend = float(df2[1][6])
        quarter_dividend = dividend / 4
        shares_needed = float("{:.2f}".format(price / quarter_dividend))
        cost = "{:,.2f}".format(price * shares_needed)
    
        print("With quarterly dividend reinvestment, you'll need about " + str(shares_needed) + " shares to purchase a new share with only dividends. This amount of shares will cost about $" + str(cost))
    
    except:
        print("Stock does not pay a dividend.")
    
    #pe = float(df2[3][0])
    
except ValueError:
    print("Ticker doesn't exist. Please check your selection.")

Hello! Today's date is: Sep 24, 2021
Time on the East Coast is currently: 12:33PM
The market is open!
Please choose a ticker symbol: Ftec


Loading data for FTEC: 100%|██████████| 100/100 [00:01<00:00, 80.84it/s]


Company/ETF name: Fidelity MSCI Information
Sector: Financial 
          Price P/E PEG P/B P/FCF Dividend Dividend % ROE
Stats:  $124.30   -   -   -     -     0.83      0.67%   -
With quarterly dividend reinvestment, you'll need about 599.04 shares to purchase a new share with only dividends. This amount of shares will cost about $74,460.67
